## Checklist for running a real WRF case 

### WRF Workflow / Steps


1. Geogrid: creates domains
    - Input from  namelis.wps (&geogrid part - and max_dom -)
    - Based on static data – the temporal part of namelist.wps does not affect geogrid
    - Creates geo_em.d01.nc (1 file per domain)

2. Ungrib: Unpacks GRIB files for boundary and initial conditions
    - Input from  namelis.wps (&ungrib part)
    - Requires the correct variable table (“Vtable”) translator
    - Creates intermediate files called FILE:* or the prefix you have on  namelist.wps -> prefix = 'FILE'

3. Metgrid: Interpolates horizontally intermediate files (FILE:*) from ungrib to WRF grid defined in geogrid
    - Input from namelis.wps (&metgrib part) fg_name = 'FILE’
    - Creates files met_em.d01.* (multiple files per domain)

4. Real: Interpolates vertically metgrid files into WRF’s vertical coordinate
    - Input from namelist.input (&domains section)
    - Creates wrfinput_d01 and wrfbdy_d01 (1 wrf_input per domain, 1 wrfbdy usually). Sometimes creates (wrflowinp_d0* 1 per domain)

5. WRF: Produces the forecast / simulation
    - Input from namelist.input (all sections)
    - Needs input files from real
    - Creates output files wrfout_d01_2023-01-22_12:00:00 (for each domain)


##### NOTE: 

If you use ERA5 as boundary conditions using https://github.com/NCAR/era5_to_int you don't need to run ungrib (era5_to_int replaces it)


### General / Basic

1. Create a directory for your project

    - `mkdir newproject` -> creates a directory called newproject

    - `cd newproject` -> change directory to newproject

2. copy WPS and WRF.

    - `cp -r ~wrfhelp/derecho_pre_compiled_code/wrfv4.7.1 WRF`

    - `cp -r ~wrfhelp/derecho_pre_compiled_code/wpsv4.6.0 WPS`


    - You now have inside `newproject` (the name you used) two directories called `WPS` and `WRF`

3. go to WPS and confirm you have geogrid.exe, ungrib.exe, and metgrid.exe

    - `cd WPS` 

    - `ls *.exe`

    - you should see: `geogrid.exe  metgrid.exe  ungrib.exe`

    - copy the running scripts (to submit jobs to the queue)

    - `cp /glade/work/jhenao/atm746/WPS/run*.sh .`



4. go to WRF/test/em_real and confirm you have `real.exe` and `wrf.exe`
   
    - `cd ../WRF/test/em_real/` 

    - `ls *.exe`

    - you should see: `ndown.exe  real.exe  tc.exe  wrf.exe`

    - copy the running scripts

    - `cp /glade/derecho/scratch/jhenao/atm746/ar_test/run*.sh .`

 




### WPS

It is controlled mostly using the `namelist.wps`

When using multiple domains, some inputs need to have one column per domain (max_dom)

1. For Geogrid

    - Edit `max_dom` in `&share` section of namelist to the number of domains

    - Edit `&geogrid` section of namelist.wps and use options from domain wizard or the way you defined your domains

        - use `geog_data_path = '/glade/work/wrfhelp/WPS_GEOG/'` (this works in NCAR only)

        - additional details in https://github.com/jjhenao/laboratory_atm746/blob/main/2.1_run_real_wrf_case.ipynb

        - run geogrid.exe `qsub rungeogrid.sh`

        - check the status `qstat -u $USER`

        - after geogrid.exe runs, if it is successfull:

            - `tail geogrid.log` -> you should see `*** Successful completion of program geogrid.exe ***`

            - `ls -lh geo_em.d0*`  -> you should see one file per domain

2. For Ungrib
    - Edit `&share` section of namelist.wps for `start_date`, `end_date`, and `interval_seconds`

    - Link the Vtable to be used to the `WPS` directory, this depends on the data for boundary conditions

        - available tables are located in `ungrib/Variable_Tables/`.

        - we used `Vtable.GFS` as we used FNL which shares the Vtable with GFS
    - In `&ungrib` section `prefix=’FILE’`determines the name of files created by ungrib

    - link your met files using `./link_grib.csh`
        - `ls -l GRIBFILE*` -> you should see multiple `GRIBFILE*` files

    - you can run ungrib.exe `qsub runungrib.sh` (**use 1 cpu only**)

        - you can check the status using `qstat -u $USER`

        - after ungrib.exe runs, if it is successfull:
            - `tail ungrib.log` -> you should see `*** Successful completion of program ungrib.exe ***`

            - `ls -lh FILE*`  -> you should see multiple `FILE:*` files

2. For Metgrid (both geogrid.exe & ungrib.exe should be succesfull)

    - In `&metgrid` section `fg_name` indicates the prefix of intermediate files (usually `FILE` or what is on `prefix`)

    - you can run metgrid.exe `qsub runmetgrid.sh`

        - you can check the status using `qstat -u $USER`

        - after metgrid.exe runs, if it is successfull:

            - `tail metrid.log` -> you should see `*** Successful completion of program metgrid.exe ***`

            - `ls -lh met_em.d0*`  -> you should see multiple files per domain


### WRF

It is controlled mostly using the `namelist.input`

When using multiple domains, some inputs need to have one column per domain (max_dom)

**make sure you have at least the number of columns you need for those inputs**, you can have more and nothing happens but if you have less the **default** option is selected and may cause problems

Check here if a field is `single entry` (one column for all domains) or `max_dom` (one column per domain): https://www2.mmm.ucar.edu/wrf/users/wrf_users_guide/build/html/namelist_variables.html


1. For Real

    - move or link your `met_em*` files to the WRF directory, they are needed to run real.exe

    - Edit `namelist.input`
        - edit start and end times (run_days, hours, seconds is optional, but is the actual run time if different to 0)
        - set `history_interval` and `frames_per_outfile` to match the desired output frequency and number of files
        - set `restart_interval` to something convenient for your simulation (12 h max wall time in Derecho)
        - make sure domain settings and times match the values of WPS
        - set `time_step` between 3-6 times the DX in km of the parent domain 
        - `p_top_requested` no lower (in the atmosphere) than 5000 Pa (50hPa). Sometimes 10 hPa, but depends on the MET data.
        - `num_metgrid_levels` and `num_metgrid_soil_levels` depend on the MET data used (FNL, GFS, ERA5, etc)
        - choose physics options

    - you can run real.exe `qsub runreal.sh`

        - you can check the status using `qstat -u $USER`

        - when real.exe is running:

            - `ls rsl.error.00*` -> you will see one file per cpu
        
        - after real.exe runs, if it is successfull:
        
            - `tail rsl.error.0000` -> you should see `real_em: SUCCESS COMPLETE REAL_EM INIT`

            - `ls -lh wrfinput_*`  -> you should see one file per domain

            - `ls -l wrfbdy_d01` -> you should see one file

2. For WRF
    
    - If real.exe was successfull:

    - you can run wrf.exe `qsub runwrf.sh`

        - you can check the status using `qstat -u $USER`

        - when erf.exe is running:

            - `ls rsl.error.00*` -> you will see one file per cpu
        
        - after erf.exe runs, if it is successfull:
        
            - `tail rsl.error.0000` -> you should see `wrf: SUCCESS COMPLETE WRF`

            - `ls -lh wrfout_d*`  -> you should see multiple files per domain (frequency depends on your namelist.input selections)

            - Your simulation is complete. 
            
            - Make sure the last wrfout* file contains your end_date